In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Quality/inpatient_specialty_qual.csv').iloc[:,1:]

In [3]:
inpatient_specialty

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
0,100399,84.470360,2008-12-27 22:00:08.0,2009-01-01 01:41:45.0,"PULMONARY, NON-TB",Minnesota
1,100399,85.551411,2010-01-27 00:58:53.0,2010-02-03 20:55:56.0,SUBSTANCE ABUSE TRMT UNIT,Minnesota
2,100399,86.335481,2010-11-09 14:50:30.0,2010-11-10 17:47:52.0,SURGICAL OBSERVATION,Minnesota
3,100694,72.493945,2004-08-04 12:53:09.0,2004-08-07 14:21:19.0,SURGICAL STEPDOWN,Idaho
4,102468,59.676888,2011-04-08 21:39:44.0,2011-04-12 23:58:20.0,GEM ACUTE MEDICINE,North Carolina
...,...,...,...,...,...,...
11384,98416,73.041255,2000-09-25 13:53:23.0,2000-09-26 05:09:15.0,HALFWAY HOUSE,California
11385,98416,78.018555,2005-09-18 18:45:38.0,2005-09-18 18:45:38.0,CARDIOLOGY,California
11386,99024,68.288449,2014-08-25 10:16:36.0,2014-08-25 10:16:36.0,GENERAL(ACUTE MEDICINE),Oregon
11387,9929,72.785598,2011-11-05 03:15:59.0,2011-11-07 03:15:58.0,ANESTHESIOLOGY,Arizona


In [4]:
pd.isnull(inpatient_specialty).sum() 

Internalpatientid       0
Age at specialty        0
Specialty start date    0
Specialty end date      3
Specialty               0
State                   0
dtype: int64

In [5]:
inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
4825,21909,64.164001,2021-09-30 22:37:08.0,NaN,PERIPHERAL VASCULAR,Virginia
7322,156832,71.727511,2022-03-21 09:43:30.0,NaN,NON-DOD BEDS IN VA FACILITY,North Carolina
8605,75275,74.246135,2023-11-24 23:05:58.0,NaN,DOMICILIARY PTSD,Rhode Island


# Preprocess Time
Calculate period mean and std values.

In [6]:
inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])][inpatient_specialty['Internalpatientid'] == 98424]

/tmp/ipykernel_281080/1186491515.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])][inpatient_specialty['Internalpatientid'] == 98424]


,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State


In [7]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [8]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

In [9]:
period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

In [10]:
period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

# Preprocess Specialty
Then consider the specialty, since it has a lot of different values, we want to simplify it in to several classes.
By studying the values, we can have the classify it into Medical Specialties (encoded as 0) and Rehabilitation and Support Programs (encoded as 1). And finally impute the no specific value (3.75%) with imputate_nan_binary.

In [11]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [12]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

In [13]:
specialty_count = inpatient_specialty.groupby('Internalpatientid')['Specialty'].count().reset_index()

In [14]:
specialty_count = specialty_count.rename(columns={'Specialty': 'specialty count'})

In [15]:
specialty_medical_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(0).sum()).reset_index()

In [16]:
specialty_support_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(1).sum()).reset_index()

In [17]:
specialty_medical_count = specialty_medical_count.rename(columns={'Specialty encoded': 'specialty medical count'})
specialty_support_count = specialty_support_count.rename(columns={'Specialty encoded': 'specialty support count'})

In [18]:
pd.isnull(period_std).sum() 

Internalpatientid    0
period std           0
dtype: int64

In [19]:
period_std[pd.isnull(period_std['period std'])]

,Internalpatientid,period std


In [21]:
period_mean[pd.isnull(period_mean['period mean'])]

,Internalpatientid,period mean


# Preprocess Age

In [22]:
age_mean = inpatient_specialty.groupby('Internalpatientid')['Age at specialty'].mean().reset_index()
age_mean = age_mean.rename(columns={'Age at specialty': 'mean age at specailty'})

# Merge All

In [23]:
listTables = [period_mean, period_std, specialty_medical_count,specialty_support_count, specialty_count ]
inpatient_specialty_preprocessed = age_mean.copy()
for i in listTables:
    inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.merge(i,how = 'left', on = 'Internalpatientid')


In [24]:
inpatient_specialty_preprocessed

,Internalpatientid,mean age at specailty,period mean,period std,specialty medical count,specialty support count,specialty count
0,67,56.998234,0.089728,0.089728,2,0,2
1,200,79.664650,1.771890,3.276867,9,6,15
2,291,82.644124,3.711998,5.827537,7,15,23
3,330,70.947996,1.829252,1.986030,14,1,15
4,351,84.981808,2.304762,2.520672,4,12,16
...,...,...,...,...,...,...,...
627,166881,75.529507,1.937271,2.161738,4,0,4
628,167102,68.142469,1.591543,1.560624,6,3,9
629,167404,77.965193,3.875710,3.145414,2,0,3
630,167917,45.426367,0.803098,0.899967,13,0,14


In [25]:
inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.dropna(subset=['period mean'])

In [26]:
inpatient_specialty_preprocessed.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_cvd_mortality_quality.csv')

In [39]:
pd.isnull(inpatient_specialty_preprocessed).sum()

Internalpatientid          0
mean age at specailty      0
period mean                0
period std                 0
specialty medical count    0
specialty support count    0
specialty count            0
dtype: int64